# Advanced Thyroid Cancer Data Analysis

## Analysis Sections:
1. Advanced Statistical Tests
2. Time Series and Age-based Analysis
3. Multivariate Analysis
4. Clinical Correlations
5. Risk Factor Interactions
6. Demographic Deep Dive
7. Clustering Analysis
8. Advanced Visualizations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = [12, 8]

## 1. Data Loading and Initial Setup

In [ ]:
# Load the data
df = pd.read_csv('thyroid_cancer_risk_data.csv')

# Define variable categories
numerical_vars = ['Age', 'TSH_Level', 'T3_Level', 'T4_Level', 'Nodule_Size']
categorical_vars = ['Gender', 'Country', 'Ethnicity', 'Family_History', 
                   'Radiation_Exposure', 'Iodine_Deficiency', 'Smoking', 
                   'Obesity', 'Diabetes']

# Create age groups
df['Age_Group'] = pd.cut(df['Age'], bins=[0, 30, 45, 60, 75, 100], 
                        labels=['<30', '30-45', '45-60', '60-75', '>75'])

## 2. Advanced Statistical Analysis

In [ ]:
# ANOVA test for clinical measurements across age groups
print("ANOVA Tests for Clinical Measurements across Age Groups:")
for var in numerical_vars:
    f_stat, p_val = stats.f_oneway(*[group[var].values 
                                    for name, group in df.groupby('Age_Group')])
    print(f"\n{var}:")
    print(f"F-statistic: {f_stat:.4f}")
    print(f"p-value: {p_val:.4e}")
    
    # Post-hoc Tukey test
    tukey = pairwise_tukeyhsd(df[var], df['Age_Group'])
    print("\nTukey's HSD test results:")
    print(tukey)

# Chi-square tests for categorical variables
print("\nChi-square Tests for Independence:")
for var1 in categorical_vars:
    for var2 in categorical_vars[categorical_vars.index(var1)+1:]:
        contingency = pd.crosstab(df[var1], df[var2])
        chi2, p_val, dof, expected = stats.chi2_contingency(contingency)
        print(f"\n{var1} vs {var2}:")
        print(f"Chi-square statistic: {chi2:.4f}")
        print(f"p-value: {p_val:.4e}")

## 3. Clinical Measurements Deep Dive

In [ ]:
# Advanced clinical measurements analysis
# Create composite scores
df['Thyroid_Function_Score'] = (
    (df['TSH_Level'] - df['TSH_Level'].mean()) / df['TSH_Level'].std() +
    (df['T3_Level'] - df['T3_Level'].mean()) / df['T3_Level'].std() +
    (df['T4_Level'] - df['T4_Level'].mean()) / df['T4_Level'].std()
)

# Create interactive 3D scatter plot
fig = px.scatter_3d(df, x='TSH_Level', y='T3_Level', z='T4_Level',
                    color='Diagnosis', size='Nodule_Size',
                    hover_data=['Age', 'Gender', 'Country'],
                    title='3D Visualization of Thyroid Function Tests')
fig.show()

# Correlation heatmap with clustering
clinical_data = df[numerical_vars]
correlation = clinical_data.corr()

# Generate clustered heatmap
plt.figure(figsize=(12, 10))
sns.clustermap(correlation, annot=True, cmap='coolwarm', center=0)
plt.title('Clustered Correlation Heatmap of Clinical Measurements')
plt.show()

## 4. Risk Factor Interaction Analysis

In [ ]:
# Create risk interaction terms
risk_factors = ['Family_History', 'Radiation_Exposure', 'Iodine_Deficiency', 
                'Smoking', 'Obesity', 'Diabetes']

# Calculate risk interactions
for i, factor1 in enumerate(risk_factors):
    for factor2 in risk_factors[i+1:]:
        interaction_name = f"{factor1}_{factor2}_Interaction"
        df[interaction_name] = ((df[factor1] == 'Yes') & (df[factor2] == 'Yes')).astype(int)

# Analyze risk combinations
df['Risk_Count'] = df[risk_factors].apply(lambda x: (x == 'Yes').sum(), axis=1)

# Plot risk interaction patterns
plt.figure(figsize=(15, 8))
sns.boxplot(data=df, x='Risk_Count', y='Thyroid_Function_Score', hue='Diagnosis')
plt.title('Impact of Multiple Risk Factors on Thyroid Function')
plt.show()

# Create network graph of risk factor interactions
import networkx as nx

G = nx.Graph()
for factor in risk_factors:
    G.add_node(factor)

for i, factor1 in enumerate(risk_factors):
    for factor2 in risk_factors[i+1:]:
        interaction = ((df[factor1] == 'Yes') & (df[factor2] == 'Yes')).mean()
        if interaction > 0.1:  # Only show strong interactions
            G.add_edge(factor1, factor2, weight=interaction)

plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=2000, font_size=10, font_weight='bold')
plt.title('Risk Factor Interaction Network')
plt.show()

## 5. Geographic and Demographic Patterns

In [ ]:
# Calculate country-specific statistics
country_stats = df.groupby('Country').agg({
    'Diagnosis': lambda x: (x == 'Malignant').mean(),
    'Age': 'mean',
    'TSH_Level': 'mean',
    'Risk_Count': 'mean'
}).round(3)

# Create sunburst chart for hierarchical demographic analysis
fig = px.sunburst(df, path=['Country', 'Ethnicity', 'Gender'],
                  color='Risk_Count',
                  title='Hierarchical View of Demographics and Risk Factors')
fig.show()

# Create demographic risk profiles
demographic_risk = df.groupby(['Country', 'Ethnicity', 'Age_Group'])['Diagnosis'].agg([
    ('Malignant_Rate', lambda x: (x == 'Malignant').mean()),
    ('Count', 'count')
]).reset_index()

# Plot demographic risk patterns
fig = px.scatter(demographic_risk, 
                x='Count', y='Malignant_Rate',
                size='Count',
                color='Country',
                facet_col='Age_Group',
                hover_data=['Ethnicity'],
                title='Demographic Risk Patterns by Age Group')
fig.show()

## 6. Advanced Clustering Analysis

In [ ]:
# Prepare data for clustering
cluster_vars = numerical_vars + ['Risk_Count']
X = df[cluster_vars].copy()
X_scaled = StandardScaler().fit_transform(X)

# Perform PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Plot explained variance ratio
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1),
         np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('PCA Explained Variance Ratio')
plt.show()

# K-means clustering
n_clusters = range(2, 11)
silhouette_scores = []

for n in n_clusters:
    kmeans = KMeans(n_clusters=n, random_state=42)
    cluster_labels = kmeans.fit_predict(X_scaled)
    silhouette_avg = silhouette_score(X_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# Plot silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(n_clusters, silhouette_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.show()

# Perform final clustering with optimal number of clusters
optimal_clusters = n_clusters[np.argmax(silhouette_scores)]
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_scaled)

# Analyze cluster characteristics
cluster_profile = df.groupby('Cluster').agg({
    'Age': 'mean',
    'TSH_Level': 'mean',
    'T3_Level': 'mean',
    'T4_Level': 'mean',
    'Nodule_Size': 'mean',
    'Risk_Count': 'mean',
    'Diagnosis': lambda x: (x == 'Malignant').mean()
}).round(3)

print("\nCluster Profiles:")
display(cluster_profile)

## 7. Time-Series and Age-based Analysis

In [ ]:
# Create age-based analysis
age_analysis = df.groupby('Age_Group').agg({
    'TSH_Level': ['mean', 'std'],
    'T3_Level': ['mean', 'std'],
    'T4_Level': ['mean', 'std'],
    'Nodule_Size': ['mean', 'std'],
    'Diagnosis': lambda x: (x == 'Malignant').mean()
}).round(3)

# Plot age-based trends
fig = go.Figure()
for measure in ['TSH_Level', 'T3_Level', 'T4_Level', 'Nodule_Size']:
    fig.add_trace(go.Scatter(
        x=age_analysis.index,
        y=age_analysis[measure]['mean'],
        name=measure,
        error_y=dict(type='data', array=age_analysis[measure]['std'])
    ))

fig.update_layout(title='Clinical Measurements by Age Group',
                 xaxis_title='Age Group',
                 yaxis_title='Value')
fig.show()

# Create age-specific risk profiles
age_risk_profile = pd.pivot_table(df,
                                 values='Diagnosis',
                                 index='Age_Group',
                                 columns=['Gender', 'Family_History'],
                                 aggfunc=lambda x: (x == 'Malignant').mean())

# Plot age-specific risk heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(age_risk_profile, annot=True, fmt='.3f', cmap='YlOrRd')
plt.title('Age-specific Risk Profiles by Gender and Family History')
plt.show()

## 8. Survival Analysis

In [ ]:
from lifelines import KaplanMeierFitter

# Create pseudo-survival data (using Age as time variable)
df['Event'] = (df['Diagnosis'] == 'Malignant').astype(int)

# Fit KM model for different risk groups
kmf = KaplanMeierFitter()

plt.figure(figsize=(12, 8))
for risk_level in ['High', 'Medium', 'Low']:
    mask = df['Thyroid_Cancer_Risk'] == risk_level
    kmf.fit(df[mask]['Age'],
            df[mask]['Event'],
            label=risk_level)
    kmf.plot()

plt.title('Survival Curves by Risk Level')
plt.xlabel('Age')
plt.ylabel('Survival Probability')
plt.show()

# Log-rank test
from lifelines.statistics import logrank_test

high_risk = df[df['Thyroid_Cancer_Risk'] == 'High']
low_risk = df[df['Thyroid_Cancer_Risk'] == 'Low']

results = logrank_test(high_risk['Age'], low_risk['Age'],
                      high_risk['Event'], low_risk['Event'])

print('\nLog-rank test results:')
print(f'p-value: {results.p_value:.4e}')